In [1]:
import pprint
import operator
import itertools
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
def flatten(container):
    for i in container:
        if isinstance(i, (list, tuple)):
            yield from flatten(i)
        else:
            yield i

In [3]:
def calculate_prob(y, prob_class):
    n_class = sum(prob_class == i for i in flatten(y))

    return n_class

In [4]:
def format_input(X):
    if not isinstance(X, (pd.DataFrame, pd.Series)):
        if isinstance(X, dict):
            X = pd.DataFrame(X)
        else:
            if len(X.shape) == 1:
                X = X.reshape(-1, 1)
            X = pd.DataFrame(X,
                             columns=['x' + str(i) for i in range(X.shape[1])])

    return X

In [5]:
def get_class_counts(y):
    classes = np.unique(y)
    counts = []
    class_counts = {}

    for c in classes:
        counts.append(int(sum(c == i for i in flatten(y.values))))
        class_counts[c] = int(sum(c == i for i in flatten(y.values)))

    return class_counts

In [6]:
def get_datatypes(X):
    datatypes = {}
    for name, dtype in X.dtypes.items():
        if dtype == 'O':
            datatypes[name] = 'categorical'
        else: 
            datatypes[name] = 'numerical'
            
    return datatypes

In [7]:
def calculate_entropy(y):
    class_counts = get_class_counts(y)
    class_entropy = {
        cl: -count / sum(class_counts.values()) *
        np.log2(count / sum(class_counts.values()))
        for cl, count in class_counts.items()
    }
    entropy = sum(class_entropy.values())

    return entropy

In [8]:
def get_split_condition(X, col):

    global datatypes

    if datatypes[X.columns[col]] == 'categorical':
        values = X.iloc[:, col].unique()
    else:
        X_sorted = X.sort_values(X.columns[col])
        val = list(set(X_sorted.iloc[:, col].values))
        if len(val) < 2:
            values = val
        else:
            values = [
                round((val[i + 1] + val[i]) / 2, 4)
                for i in range(len(val) - 1)
            ]

    return values

In [9]:
def get_subset(X, col, val):

    global datatypes

    if datatypes[X.columns[col]] == 'categorical':
        eq_X = X.loc[X[X.columns[col]] == val]
    else:
        eq_X = X.loc[X[X.columns[col]] < val]

    not_eq_X = X.drop(eq_X.index)

    return eq_X, not_eq_X

In [10]:
def is_pure(y):
    return len(np.unique(y.values)) == 1

In [11]:
def build_tree(X,
               y,
               max_depth=None,
               min_samples=None,
               tree=None):

    global datatypes
    
    if tree is None:
        tree = {}

    entropies = {}
    columns = {}

    for col_idx in range(X.shape[1]):

        split_vals = get_split_condition(X, col_idx)

        for val in split_vals:
            eq_X, not_eq_X = get_subset(X, col_idx, val)
            eq_y, not_eq_y = y.loc[eq_X.index, :], y.loc[not_eq_X.index, :]

            eq_entr = calculate_entropy(eq_y)
            not_eq_entr = calculate_entropy(not_eq_y)

            avg_entr = (len(eq_y) / len(y) * eq_entr) + (len(not_eq_y) /
                                                         len(y) * not_eq_entr)

            entropies[val] = avg_entr
            columns[val] = col_idx

    best_split_key = min(entropies, key=entropies.get)
    best_split_value = entropies.get(best_split_key)

    eq_X, not_eq_X = get_subset(X, columns[best_split_key], best_split_key)
    eq_y, not_eq_y = y.loc[eq_X.index, :], y.loc[not_eq_X.index, :]

    print('Best split', best_split_key, best_split_value)

    column_name = X.columns[columns[best_split_key]]

    if datatypes[column_name] == 'categorical':
        condition = column_name + ' == ' + best_split_key
    else:
        condition = column_name + ' < ' + str(best_split_key)

    tree[condition] = []
    if is_pure(eq_y):
        tree[condition].append(np.unique(eq_y)[0])
    else:
        tree[condition].append(build_tree(eq_X, eq_y))

    if is_pure(not_eq_y):
        tree[condition].append(np.unique(not_eq_y)[0])
    else:
        tree[condition].append(build_tree(not_eq_X, not_eq_y))

    return tree

In [12]:
def get_prediction(instance, tree):

    node = list(tree.keys())[0]

    operators = {'==': operator.eq, '<': operator.le}

    col, op, val = node.split(' ')

    if instance.dtypes[col] == 'O':
        compare_eq = operators[op](instance[col].values[0], val)
    else:
        compare_eq = operators[op](instance[col].values[0], float(val))

    if compare_eq:
        ans = tree[node][0]
    else:
        ans = tree[node][1]

    if not isinstance(ans, dict):
        return ans
    else:
        return get_prediction(instance, ans)

In [13]:
def predict(X, tree):

    predictions = []

    for i in range(len(X)):
        predictions.append(get_prediction(X.iloc[[i], :], tree))

    return predictions

In [14]:
def score(X, y, tree):
    correct = 0

    for i in range(len(X)):
        pred = predict(X.iloc[[i]], tree)
        if pred == y.values[i]:
            correct += 1

    return correct / len(X) * 100

### Dataset with categorical features

In [15]:
X = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard']}
       
y = {'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

X = format_input(X)
y = format_input(y)

In [16]:
datatypes = get_datatypes(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [18]:
tree = build_tree(X_train, y_train)

Best split Sweet 0.75
Best split Cold 0.9182958340544896
Best split Spicy 0.6666666666666666
Best split Soft 0.0
Best split Soft 0.0


In [19]:
pprint.pprint(tree, width=50)

{'Taste == Sweet': ['Yes',
                    {'Temperature == Cold': [{'Taste == Spicy': [{'Texture == Soft': ['Yes',
                                                                                      'No']},
                                                                 'No']},
                                             {'Texture == Soft': ['No',
                                                                  'Yes']}]}]}


In [20]:
print('Train Accuracy:', round(score(X_train, y_train, tree), 2))
print('Test Accuracy:', round(score(X_test, y_test, tree), 2))

Train Accuracy: 100.0
Test Accuracy: 100.0


### Dataset with continuous features

In [21]:
X, y = load_iris(return_X_y=True)

X = format_input(X)
y = format_input(y)

In [22]:
datatypes = get_datatypes(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [24]:
tree = build_tree(X_train, y_train)

Best split 2.45 0.6898549587177165
Best split 1.75 0.31721308476515864
Best split 5.0 0.2483500540348903
Best split 1.65 0.0
Best split 2.65 0.5509775004326937
Best split 5.45 0.0
Best split 4.9 0.05555555555555555
Best split 6.05 0.0


In [25]:
pprint.pprint(tree, width=50)

{'x2 < 2.45': [0,
               {'x3 < 1.75': [{'x2 < 5.0': [{'x3 < 1.65': [1,
                                                           2]},
                                            {'x1 < 2.65': [2,
                                                           {'x2 < 5.45': [1,
                                                                          2]}]}]},
                              {'x2 < 4.9': [{'x0 < 6.05': [1,
                                                           2]},
                                            2]}]}]}


In [26]:
print('Train Accuracy:', round(score(X_train, y_train, tree), 2))
print('Test Accuracy:', round(score(X_test, y_test, tree), 2))

Train Accuracy: 98.33
Test Accuracy: 90.0
